In [5]:
# Clear workspace
rm(list=ls())

In [6]:
# Garabage collect to help with memory issues
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1444147,77.2,2371302,126.7,2371302,126.7
Vcells,2416122,18.5,8388608,64.0,4748694,36.3


In [7]:
# Uncomment and run to install packages if needed
# install.packages("tidyverse")
# install.packages("cluster") 
# install.packages("forecast") 
# install.packages("Metrics")
# install.packages("tseries")
# install.packages("scoringutils")

In [8]:
# Load libraries
library(tidyverse)
library(parallel)
library(cluster)
library(forecast)
library(Metrics)
library(tseries)
library(scoringutils)

# Read In and Subset Data

In [9]:
# Read in all csv files into a list of data frames
fnames <- list.files("Data/Unseen Sensor/Processed/", pattern="*.csv", full.names=TRUE)
total_df_list <- lapply(fnames, read_csv)

Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): site_name
dbl  (5): day_of_week, day_of_year, interval_of_day, avg_mph, total_volume
lgl  (2): missing_speed, missing_volume
dttm (1): timestamp
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): site_name
dbl  (5): day_of_week, day_of_year, interval_of_day, avg_mph, total_volume
lgl  (2): missing_speed, missing_volume
dttm (1): timestamp
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 35040 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delim

In [10]:
# Ensure all data frames are in proper time series order
total_df_list <- lapply(total_df_list, function(x) x %>% arrange(timestamp))    

In [11]:
# Read in the start and end points for each time series from csv
start_end <- read_csv("start_end_points_unseen.csv")
starting_points <- start_end$start
ending_points <- start_end$end

Rows: 4 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [12]:
# For each df in our list, create a row number column called 'rn' - this will allow us to find the starting point
# for each sample using the above starting_points array
total_df_list <- lapply(total_df_list, function(x) x %>% mutate(rn = row_number()))

In [13]:
# Sample each df in the list according to its corresponding starting and ending point - this creates 12-week
# long samples of each data frame which will be used for modeling and testing
total_df_list_samples <- lapply(1:length(total_df_list), 
                                function(x) total_df_list[[x]] %>% 
                                    filter(rn >= starting_points[x]) %>%
                                    filter(rn <= ending_points[x]))

In [14]:
# Add a new column to each data frame to denote whether each row should be part of the training set, validation set
# or test set - the first 8 weeks are designated as train, the next 2 as val, and the final 2 as test. It may be 
# the case that train and val get used for training, depending on the modeling task
total_df_list_samples <- lapply(total_df_list_samples, 
                                function(x) x %>% mutate(rn = row_number()) %>%
                                    mutate(train_val_test = ifelse(rn <= (96*7*8), 
                                                                   "train", 
                                                                   ifelse(rn <= (96*7*10), 
                                                                          "val", 
                                                                          "test")))
                                
                               )

In [15]:
# Create a list of data frames which only have the training and validation rows of each df
train_val_samples <- lapply(total_df_list_samples, function(x) x %>% filter(train_val_test != "test"))

In [16]:
# Create a list of data frames which only contain the test samples for each df
test_samples <- lapply(total_df_list_samples, function(x) x %>% filter(train_val_test == "test"))

In [17]:
# Create a list of time series objects from the total_volume column of each data frame using the msts function
# This function allows for time series with multiple seasonality, so we use both daily and weekly seasonal periods
# Having time series objects instead of data frame columns is helpful for some tasks
# Here, we create this list for the train_val samples list, next for the test samples
train_val_samples_ts <- lapply(train_val_samples, 
                               function(x) msts(x$total_volume, seasonal.periods=c(24*4, 24*4*7)))

In [18]:
# Create a similar list for the test data
test_samples_ts <- lapply(test_samples,
                          function(x) msts(x$total_volume, seasonal.periods=c(24*4, 24*4*7)))

In [19]:
# Set the random seed for reproducibility
set.seed(54321)

# ARIMA with Fourier Terms

In [20]:
arima_w_fourier <- function(ts_train) { 
    # Function which takes in training time series object, creates Fourier terms for that ts,  
    # and returns a model using auto.arima and fourier terms as regressors
    
    # Create Fourier terms with K=2 for both frequencies/seasonal periods
    fourier_term_train <- fourier(ts_train, K=c(2,2))
    
    # Use auto.arima with default parameters and seasonal set to FALSE 
    # as the Fourier terms account for the seasonality
    arima_model <- auto.arima(ts_train, xreg = fourier_term_train, seasonal = FALSE) 
    
    # Return the model from auto.arim
    arima_model
}

In [21]:
# Call the above arima_w_fourier function for each train_val sample in our data set
# Because we are allowing auto.arime to choose the final modeling structure for each time series, we
# are passing the full train and validation time series together. We could also manually compute the best
# model structure by creating models on the training time frames and computing forecast accuracy on the validation
# time frames. It would be an interesting avenue of future work to test whether this makes any difference 
# in terms of model performance on the final test sets
arima_res <- lapply(1:length(train_val_samples_ts), 
                    function(x) arima_w_fourier(train_val_samples_ts[[x]]))

In [22]:
arima_w_fourier_predict <- function(ts_train, ts_test, model) {
    # Function to make one-step ahead forecasts given a model, the data the model was trained on, and testing data
    
    # Create the fourier terms for training and testing sets
    fourier_term_train <- fourier(ts_train, K=c(2,2))
    fourier_term_test <- fourier(ts_train, K=c(2,2), h=length(ts_test))

    # Make one-step ahead forecasts on the training data using the fitted function
    train_preds <- fitted(model)
    
    # Compute the rmse and mae of the one-step ahead predictions on the training data
    train_rmse <- rmse(ts_train, train_preds)
    train_mae <- mae(ts_train, train_preds)
    
    # Create a "test" version of the model by adding in the test data and calling Arima with the full data set
    # and the already constructed model
    mod_test <- Arima(y=c(ts_train, ts_test), 
                      xreg=rbind(fourier_term_train,
                                 fourier_term_test),
                      model=model)
    
    # Compute test preds by calling fitted on the new model and eliminating the training preds component of the 
    # results
    test_preds <- fitted(mod_test)[-c(1:length(ts_train))]
    
    # Compute the rmse and mae on the one-step ahead forecasts on the test data
    test_rmse <- rmse(ts_test, test_preds)
    test_mae <- mae(ts_test, test_preds)
    
    # Return a list of the rmse and mae for train and test sets, as well as the actual forecasts on the test data
    return_list <- list("train_rmse"=train_rmse, "train_mae"=train_mae,
                        "test_rmse"=test_rmse, "test_mae"=test_mae, "raw_forecasts"=test_preds)
}

In [23]:
# Call the above function for each time series in our data set and its corresponding ARIMA model
arima_perf <- lapply(1:length(test_samples_ts),
                     function(x) arima_w_fourier_predict(train_val_samples_ts[[x]],
                                                         test_samples_ts[[x]],
                                                         arima_res[[x]])
                    )

In [25]:
# Create a data frame from the above results, with one column for each train and test rmse and mae
arima_results <- data.frame(train_rmse=unlist(lapply(arima_perf, function(x) x$train_rmse)),
                            train_mae=unlist(lapply(arima_perf, function(x) x$train_mae)),
                            test_rmse=unlist(lapply(arima_perf, function(x) x$test_rmse)),
                            test_mae=unlist(lapply(arima_perf, function(x) x$test_mae)))

In [26]:
# In the next several cells, print and inspect the performance of each model on each time series, including
# examining rmse, mae, and the normalized/standardized versions of each by first computing the mean of the time
# series and scaling appropriately
mean(arima_results$train_rmse)

[1] 45.61953

In [27]:
mean(arima_results$train_mae)

[1] 30.97378

In [28]:
mean(arima_results$test_rmse)

[1] 45.19907

In [29]:
mean(arima_results$test_mae)

[1] 30.93976

In [30]:
# Compute means for scaling RMSE and MAE
test_means <- unlist(lapply(test_samples_ts, mean))
train_means <- unlist(lapply(train_val_samples_ts, mean))

In [31]:
mean(arima_results$train_rmse/train_means)

[1] 0.1354646

In [32]:
mean(arima_results$train_mae/train_means)

[1] 0.09378641

In [33]:
mean(arima_results$test_rmse/test_means)

[1] 0.1415735

In [34]:
mean(arima_results$test_mae/test_means)

[1] 0.09724901

In [35]:
# Create a list of just the one-step ahead forecasts for each test set
arima_raw_forecasts <- lapply(1:length(arima_perf), 
                              function(x) data.frame(forecast=arima_perf[[x]]$raw_forecasts, ts_index=x))

In [36]:
# Take that list and turn it into a data frame
arima_raw_forecasts_df <- do.call("rbind", arima_raw_forecasts)

In [37]:
# Print head of the df
head(arima_raw_forecasts_df)

,forecast,ts_index
,<dbl>,<int>
1,247.0410,1
2,241.0415,1
3,307.1620,1
4,324.4770,1
5,319.4103,1
6,274.4670,1


In [38]:
# Check df length - just a sanity check to make sure things look alright
nrow(arima_raw_forecasts_df)

[1] 5376

In [39]:
bootstrap_pred_int <- function(predictions,
                               residuals,
                               n_boot,
                               actuals=NULL
                              ) 
{
    # Function to compute prediction intervals using bootstrapping with model residuals
    # function takes in the predicted values, the model residuals, the number of bootstrap samples to use,
    # and an optional set of true values (used for computing prediction interval scores)
    
    # Intialize empty lists for the hi and lo components of the 80% nad 95% PIs
    boot_lo_80 <- c()
    boot_hi_80 <- c()
    boot_lo_95 <- c()
    boot_hi_95 <- c()

    # Loop through every row in the predictions
    for (i in 1:nrow(predictions)){
        # Reset the seed to the loop counter - could make this more random, but this is for reproducibility
        set.seed(i)
        
        # Sample with replacement from the residuals n_boot times
        boot_samples <- sample(residuals, n_boot, replace=TRUE)
        # Add the predicted value to each sample
        boot_preds <- boot_samples + predictions$forecast[i]
        
        # Compute the PIs as the quantiles of these pred+boot values
        boot_pred_lo_80 <- quantile(boot_preds, 0.1)
        boot_pred_hi_80 <- quantile(boot_preds, 0.9)

        boot_pred_lo_95 <- quantile(boot_preds, 0.025)
        boot_pred_hi_95 <- quantile(boot_preds, 0.975)
        
        # Append the PI values to the lists
        boot_lo_80 <- c(boot_lo_80, boot_pred_lo_80)
        boot_hi_80 <- c(boot_hi_80, boot_pred_hi_80)

        boot_lo_95 <- c(boot_lo_95, boot_pred_lo_95)
        boot_hi_95 <- c(boot_hi_95, boot_pred_hi_95)        
    }
    
    # Create a copy of the input df called predicitons
    pred_w_boot_int_df <- predictions
    
    # Add columns to the df for the PIs
    pred_w_boot_int_df$lo_80 <- boot_lo_80
    pred_w_boot_int_df$hi_80 <- boot_hi_80
    pred_w_boot_int_df$lo_95 <- boot_lo_95
    pred_w_boot_int_df$hi_95 <- boot_hi_95
    
    # If the true values are included
    if (!is.null(actuals)){
        
        # Add a column for the true values
        pred_w_boot_int_df$true_values <- actuals
        
        # Compute the 80% and 95% interval scores and add them as columns in the df
        pred_w_boot_int_df <- pred_w_boot_int_df %>%
            mutate(int_score_80 = interval_score(true_values=true_values,
                                                 lower=lo_80,
                                                 upper=hi_80,
                                                 interval_range=80,
                                                 weigh=FALSE
                                                )) %>% 
            mutate(int_score_95 = interval_score(true_values=true_values,
                                                 lower=lo_95,
                                                 upper=hi_95,
                                                 interval_range=95,
                                                 weigh=FALSE
                                                ))
    }
    
    # Return the final set of preds with PIs (and interval scores if applicable)
    pred_w_boot_int_df
    
}

In [40]:
# Set the number of bootstrap samples to use
n_boot <- 1000

In [41]:
# Compute bootstrap PI for each model/ts
bootstrap_pi <- lapply(1:length(arima_raw_forecasts), 
                      function(i) bootstrap_pred_int(arima_raw_forecasts[[i]], 
                                                     arima_res[[i]]$residuals, 
                                                     n_boot, 
                                                     test_samples_ts[[i]]))

In [42]:
# Print the mean of the 80% PI scores
mean(unlist(lapply(bootstrap_pi, function(x) mean(x$int_score_80))))

[1] 165.0553

In [43]:
# Print the mean of the 95% PI scores
mean(unlist(lapply(bootstrap_pi, function(x) mean(x$int_score_95))))

[1] 270.9179

In [44]:
# Print the means of the scaled interval scores
mean(unlist(lapply(bootstrap_pi, function(x) mean(x$int_score_80)))/test_means)

[1] 0.5207743

In [45]:
mean(unlist(lapply(bootstrap_pi, function(x) mean(x$int_score_95)))/test_means)

[1] 0.840633

# Naive Forecasts

In [46]:
naive_forec <- function(train_ts, test_ts, lag=1) {
    # Function to make "naive" forecasts using a given lag, and then compute the accuracy metrics and residuals
    
    # Train forecasts are simply computed as the value at the time series lag steps ago
    train_forec <- lag(train_ts, lag)
    
    # Compute the residuals on the train set, removing the first lag values because they will result in nulls
    train_res <- train_ts[-c(1:lag)] - train_forec[-c(1:lag)]
        
    # Copmute the rmse and mae on the train set
    train_rmse <- rmse(train_ts[-c(1:lag)], train_forec[-c(1:lag)])
    train_mae <- mae(train_ts[-c(1:lag)], train_forec[-c(1:lag)])
    
    # Append the final time steps of the train and test data together
    n <- length(train_ts)
    n1 <- n-(lag-1)
    train_test_data <- c(train_ts[n1:n], test_ts)
    
    # Compute test forecasts from this appended data set
    test_forec <- lag(train_test_data, lag)
    test_forec_final <- test_forec[-c(1:lag)]
        
    # Compute rmse and mae on the test data
    test_rmse <- rmse(test_ts, test_forec_final)
    test_mae <- mae(test_ts, test_forec_final)
        
    # Return a list which includes the rmse, mae for both train and test, the residuals on the train data,
    # and the raw forecasts on the test data
    return_list <- list("train_rmse"=train_rmse, "train_mae"=train_mae, "train_residuals"=train_res,
                        "test_rmse"=test_rmse, "test_mae"=test_mae, "raw_forecasts"=test_forec_final) 

}

In [47]:
bootstrap_pred_int_naive <- function(predictions,
                                     residuals,
                                     n_boot,
                                     actuals=NULL
                                    ) 
{
    # Function to compute prediction intervals using bootstrapping with model residuals
    # function takes in the predicted values, the model residuals, the number of bootstrap samples to use,
    # and an optional set of true values (used for computing prediction interval scores)
    
    # Intialize empty lists for the hi and lo components of the 80% nad 95% PIs
    boot_lo_80 <- c()
    boot_hi_80 <- c()
    boot_lo_95 <- c()
    boot_hi_95 <- c()

    # Loop through every row in the predictions
    for (i in 1:length(predictions)){
        # Reset the seed to the loop counter - could make this more random, but this is for reproducibility
        set.seed(i)
        
        # Sample with replacement from the residuals n_boot times
        boot_samples <- sample(residuals, n_boot, replace=TRUE)
        # Add the predicted value to each sample
        boot_preds <- boot_samples + predictions[i]
        
        # Compute the PIs as the quantiles of these pred+boot values
        boot_pred_lo_80 <- quantile(boot_preds, 0.1)
        boot_pred_hi_80 <- quantile(boot_preds, 0.9)

        boot_pred_lo_95 <- quantile(boot_preds, 0.025)
        boot_pred_hi_95 <- quantile(boot_preds, 0.975)
        
        # Append the PI values to the lists
        boot_lo_80 <- c(boot_lo_80, boot_pred_lo_80)
        boot_hi_80 <- c(boot_hi_80, boot_pred_hi_80)

        boot_lo_95 <- c(boot_lo_95, boot_pred_lo_95)
        boot_hi_95 <- c(boot_hi_95, boot_pred_hi_95)        
    }
    
    # Create a copy of the input df called predicitons
    pred_w_boot_int_df <- data.frame(forecast=predictions)
    
    # Add columns to the df for the PIs
    pred_w_boot_int_df$lo_80 <- boot_lo_80
    pred_w_boot_int_df$hi_80 <- boot_hi_80
    pred_w_boot_int_df$lo_95 <- boot_lo_95
    pred_w_boot_int_df$hi_95 <- boot_hi_95
    
    # If the true values are included
    if (!is.null(actuals)){
        
        # Add a column for the true values
        pred_w_boot_int_df$true_values <- actuals
        
        # Compute the 80% and 95% interval scores and add them as columns in the df
        pred_w_boot_int_df <- pred_w_boot_int_df %>%
            mutate(int_score_80 = interval_score(true_values=true_values,
                                                 lower=lo_80,
                                                 upper=hi_80,
                                                 interval_range=80,
                                                 weigh=FALSE
                                                )) %>% 
            mutate(int_score_95 = interval_score(true_values=true_values,
                                                 lower=lo_95,
                                                 upper=hi_95,
                                                 interval_range=95,
                                                 weigh=FALSE
                                                ))
    }
    
    # Return the final set of preds with PIs (and interval scores if applicable)
    pred_w_boot_int_df
    
}

## Simple Naive Forecast

In [48]:
# Apply the above function to compute simple naive forecasts - that is, lag 1 naive forecasts
simple_naive_forecasts <- lapply(1:length(train_val_samples),
                                 function(x) naive_forec(train_val_samples[[x]]$total_volume,
                                                         test_samples[[x]]$total_volume))

In [49]:
# Compute average performance across the data set
mean(unlist(lapply(simple_naive_forecasts, function(x) x$test_rmse)))

[1] 51.0199

In [50]:
mean(unlist(lapply(simple_naive_forecasts, function(x) x$test_mae)))

[1] 34.11049

In [51]:
mean(unlist(lapply(simple_naive_forecasts, function(x) x$test_rmse))/test_means)

[1] 0.1616255

In [52]:
mean(unlist(lapply(simple_naive_forecasts, function(x) x$test_mae))/test_means)

[1] 0.1090316

In [53]:
# Compute bootstrap PI for each model/ts
bootstrap_pi_naive <- lapply(1:length(simple_naive_forecasts), 
                      function(i) bootstrap_pred_int_naive(simple_naive_forecasts[[i]]$raw_forecasts,
                                                           simple_naive_forecasts[[i]]$train_res, 
                                                           n_boot, 
                                                           test_samples_ts[[i]]))

In [54]:
# Print the mean of the 80% PI scores
mean(unlist(lapply(bootstrap_pi_naive, function(x) mean(x$int_score_80))))

[1] 188.2648

In [55]:
# Print the mean of the 95% PI scores
mean(unlist(lapply(bootstrap_pi_naive, function(x) mean(x$int_score_95))))

[1] 306.7128

In [56]:
# Print the average scaled 80% PI score
mean(unlist(lapply(bootstrap_pi_naive, function(x) mean(x$int_score_80)))/test_means)

[1] 0.6012354

In [57]:
# Print the average scaled 95% PI score
mean(unlist(lapply(bootstrap_pi_naive, function(x) mean(x$int_score_95)))/test_means)

[1] 0.9555051

## Daily Seasonal Naive Forecast

In [58]:
# Use the same approach, but this time with lag=96 to use a daily naive forecast instead of a simple one
daily_naive_forecasts <- lapply(1:length(train_val_samples),
                                function(x) naive_forec(train_val_samples[[x]]$total_volume,
                                                        test_samples[[x]]$total_volume,
                                                        lag=96
                                                       )
                                )

In [59]:
# Examine model performance metrics
mean(unlist(lapply(daily_naive_forecasts, function(x) x$test_rmse)))

[1] 120.2999

In [60]:
mean(unlist(lapply(daily_naive_forecasts, function(x) x$test_mae)))

[1] 70.36868

In [61]:
# Print scaled model performance metrics
mean(unlist(lapply(daily_naive_forecasts, function(x) x$test_rmse))/test_means)

[1] 0.3469469

In [62]:
mean(unlist(lapply(daily_naive_forecasts, function(x) x$test_mae))/test_means)

[1] 0.2105529

In [63]:
# Compute bootstrap PI for each model/ts
bootstrap_pi_naive_daily <- lapply(1:length(daily_naive_forecasts), 
                      function(i) bootstrap_pred_int_naive(daily_naive_forecasts[[i]]$raw_forecasts,
                                                           daily_naive_forecasts[[i]]$train_res, 
                                                           n_boot, 
                                                           test_samples_ts[[i]]))

In [64]:
# Print the mean of the 80% PI scores
mean(unlist(lapply(bootstrap_pi_naive_daily, function(x) mean(x$int_score_80))))

[1] 437.6474

In [65]:
# Print the mean of the 95% PI scores
mean(unlist(lapply(bootstrap_pi_naive_daily, function(x) mean(x$int_score_95))))

[1] 815.2558

In [66]:
# Print scaled 80% PI score
mean(unlist(lapply(bootstrap_pi_naive_daily, function(x) mean(x$int_score_80)))/test_means)

[1] 1.284768

In [67]:
# Print scaled 95% PI score
mean(unlist(lapply(bootstrap_pi_naive_daily, function(x) mean(x$int_score_95)))/test_means)

[1] 2.270095

## Weekly Naive Forecast

In [68]:
# Again, use the same methods but this time weekly so lag=672
weekly_naive_forecasts <- lapply(1:length(train_val_samples),
                                 function(x) naive_forec(train_val_samples[[x]]$total_volume,
                                                         test_samples[[x]]$total_volume,
                                                         lag=672
                                                       )
                                )

In [69]:
# Examine RMSE
mean(unlist(lapply(weekly_naive_forecasts, function(x) x$test_rmse)))

[1] 66.1019

In [70]:
# Print average MAE
mean(unlist(lapply(weekly_naive_forecasts, function(x) x$test_mae)))

[1] 41.71633

In [71]:
# Print scaled average RMSE, or nRMSE
mean(unlist(lapply(weekly_naive_forecasts, function(x) x$test_rmse))/test_means)

[1] 0.2004922

In [72]:
# Print average scaled MAE, or sMAE
mean(unlist(lapply(weekly_naive_forecasts, function(x) x$test_mae))/test_means)

[1] 0.1261486

In [73]:
# Compute bootstrap PI for each model/ts
bootstrap_pi_naive_weekly <- lapply(1:length(weekly_naive_forecasts), 
                      function(i) bootstrap_pred_int_naive(weekly_naive_forecasts[[i]]$raw_forecasts,
                                                           weekly_naive_forecasts[[i]]$train_res, 
                                                           n_boot, 
                                                           test_samples_ts[[i]]))

In [74]:
# Print the mean of the 80% PI scores
mean(unlist(lapply(bootstrap_pi_naive_weekly, function(x) mean(x$int_score_80))))

[1] 240.9757

In [75]:
# Print the mean of the 95% PI scores
mean(unlist(lapply(bootstrap_pi_naive_weekly, function(x) mean(x$int_score_95))))

[1] 417.2614

In [76]:
# Print the average scaled 80% PI score
mean(unlist(lapply(bootstrap_pi_naive_weekly, function(x) mean(x$int_score_80)))/test_means)

[1] 0.7275303

In [77]:
# Print the average scaled 95% PI score
mean(unlist(lapply(bootstrap_pi_naive_weekly, function(x) mean(x$int_score_95)))/test_means)

[1] 1.271977